# Extraction of Laws References from the quiz questions

In [1]:
import pandas as pd
import json
import os
import re

isLinux = True
default_linux_path = os.getcwd().replace("/Data", "/Documents/Downloaded") if "/Data" in os.getcwd() else os.getcwd() + "/Documents/Downloaded"
default_windows_path = os.getcwd().replace("\\Data", "\\Documents\\Downloaded") if "\\Data" in os.getcwd() else os.getcwd() + "\\Documents\\Downloaded"
default_path = default_linux_path if isLinux else default_windows_path

DEFAULT_SAVE_DIR = default_path.replace("/Downloaded", "/Generated") if isLinux else default_path.replace("\\Downloaded", "\\Generated")
LAWS_CSV = DEFAULT_SAVE_DIR + ('/laws.csv' if isLinux else '\\laws.csv')
REFERENCES_CSV = DEFAULT_SAVE_DIR + '/references_merged.csv'
QUIZZES_CSV = DEFAULT_SAVE_DIR + '/quiz_merged.csv'

df_cp, df_cpa, df_cpp, df_cost, df_dlgs, df_laws = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

df_quiz = pd.read_csv(QUIZZES_CSV)
df_laws = pd.read_csv(LAWS_CSV)

In [108]:
def retrieve_law(source, law_id):
    
    if source == 'c.p.':        
        for i, row in df_laws.iterrows():  
            if row['law_source'] == "c.p." and law_id in row['law_number']:
                return row['law_text']
    elif source == 'c.p.p.':
        for i, row in df_laws.iterrows():
            if row['law_source'] == "c.p.p." and law_id in row['law_number']:
                return row['law_text']
    else:
        return None

    print(source, law_id)
    return None
    

def extract_data(df_quiz, regex_patterns, source):
    extracted_data = []
    no_match_data = []

    s = dict()
    for i, row in df_quiz.iterrows():
        question = row['question']
        match = None

        for pattern in regex_patterns:
            match = re.findall(pattern, question)
            question_plh = re.sub(pattern, '{PLH}', question)
            if match:
                break

        if match:
            # Search for any comma reference
            comma = re.findall(r'comma ([^ ^\.^,]+)', question)
            if comma:
                comma = comma[0].strip()
            else:
                comma = None
            
            # Catch of set of laws not single
            if source == 'D. Lgs.':
                for i, elem in enumerate(match):
                    if type(elem) is tuple:
                        reference = elem[0]
                        groupSource = elem[1].replace(' del ', '/')
                    elif type(elem) is str:
                        reference = None
                        groupSource = elem.replace(' del ', '/')
                    else:
                        raise Exception(f'Unexpected type {type(elem)}')
                    
                    extracted_data.append({
                        'law_source': groupSource.strip(),
                        'law_number': reference.strip() if reference else None,
                        'quiz_id': row['quiz_id'],
                        'question_plh': question_plh[i],
                        'law_text': retrieve_law(groupSource, reference.strip() if reference else None)
                    })
            else:
                if type(match[0]) is tuple:
                    print(source)
                reference = match[0]
                if reference.strip() in s:
                    s[reference.strip()] += 1
                else:
                    s[reference.strip()] = 1
                
                extracted_data.append({
                    'law_source': source,
                    'law_number': reference.strip() if type(reference) is tuple else reference.strip(),
                    'quiz_id': row['quiz_id'],
                    'question_plh': question_plh,
                    'law_text': retrieve_law(source, reference.strip() if type(reference) is tuple else reference.strip())
                })
        else:
            no_match_data.append(row.to_dict())

    df_cp = pd.DataFrame(extracted_data)
    df_no_match = pd.DataFrame(no_match_data)
    
    print(s)
    
    return df_cp, df_no_match

## Extraction of Penal Code's References

In [109]:
patterns = [
    r'[Aa]rtt?\.( \d+).{0,25}c\.p\.[^p^a]',
    r"(?:articolo|art\.)( ?\d+) del [Cc]odice [Pp]enale",
    r'[Cc]odice penale all\'art. (\d+)'
]

df_cp, df_quiz = extract_data(df_quiz, patterns, "c.p.")

print("CP found: ", df_cp.shape)
print("Still unmatched: ", df_quiz.shape)
df_cp.head()

c.p. 283
c.p. 295
{'240': 1, '266': 1, '24': 1, '7': 1, '19': 1, '23': 1, '28': 1, '29': 2, '31': 1, '32': 2, '47': 1, '51': 3, '56': 5, '59': 1, '61': 2, '62': 1, '65': 1, '70': 1, '99': 1, '104': 1, '109': 2, '112': 2, '120': 1, '131': 1, '133': 2, '146': 1, '147': 1, '157': 1, '158': 1, '162': 3, '163': 2, '164': 1, '168': 3, '179': 2, '189': 1, '230': 1, '361': 2, '362': 1, '339': 2, '322': 1, '375': 1, '518': 1, '576': 1, '476': 1, '477': 1, '480': 1, '495': 2, '497': 1, '349': 1, '467': 1, '471': 1, '474': 3, '483': 1, '484': 1, '116': 2, '122': 1, '123': 1, '126': 1, '128': 1, '135': 1, '137': 1, '153': 1, '154': 1, '74': 1, '170': 1, '171': 1, '172': 1, '173': 1, '174': 1, '176': 2, '178': 1, '187': 2, '188': 1, '102': 1, '111': 1, '152': 2, '202': 1, '205': 1, '209': 1, '210': 1, '213': 1, '223': 1, '319': 1, '634': 1, '221': 1, '2': 4, '39': 2, '612': 1, '635': 1, '114': 2, '5': 1, '41': 3, '43': 1, '52': 2, '650': 1, '90': 1, '357': 1, '22': 1, '640': 1, '605': 1, '101': 1, 

,law_source,law_number,quiz_id,question_plh,law_text
0,c.p.,240,224,Ai sensi dell'{PLH}Ã¨ sempre ordinata la confi...,"(Confisca) Nel caso di condanna, il giudice p..."
1,c.p.,266,225,"Ai sensi dell'{PLH}Ã¨ punito, se il fatto non ...",(Istigazione di militari a disobbedire alle le...
2,c.p.,24,226,La pena della multa ex {PLH}consiste nel pagam...,(Multa) La pena della multa consiste nel paga...
3,c.p.,7,227,Ai sensi dell'{PLH}Ã¨ punito secondo la legge ...,(Reati commessi all'estero) È punito secondo ...
4,c.p.,19,228,Tra le pene accessorie per i delitti ex {PLH}t...,(Pene accessorie: specie) Le pene accessorie ...


## Extraction of Administrative Penal Code's References

In [91]:
patterns = [
    r'[Aa]rtt?\.( \d+).{0,30}[Cc]\.p\.a[\.]?',
    r'(?:[Aa]rticolo|[Aa]rtt?\.)( \d+).{0,30} del [Cc]odice del [Pp]rocesso [Aa]mministrativo'
]

df_cpa, df_quiz_quiz = extract_data(df_quiz, patterns, "c.p.a.")

print("CPA found: ", df_cpa.shape)
print("Still unmatched: ", df_quiz.shape)
df_cpa.head()

 80
 42
 133
 7
 34
 63
 62
 30
 43
 11
 41
 133
 16
 30
 54
 25
 63
 30
 35
 11
 133
 15
 29
 35
 8
 89
 13
 53
 15
 33
 73
 6
 73
 6
 23
 9
 53
 37
 30
 5
 27
 11
 14
 35
 74
 70
 34
 45
 35
 81
 31
 102
 28
 46
 91
 73
 92
 31
{'80': 1, '42': 1, '133': 3, '7': 1, '34': 2, '63': 2, '62': 1, '30': 4, '43': 1, '11': 3, '41': 1, '16': 1, '54': 1, '25': 1, '35': 4, '15': 2, '29': 1, '8': 1, '89': 1, '13': 1, '53': 2, '33': 1, '73': 3, '6': 2, '23': 1, '9': 1, '37': 1, '5': 1, '27': 1, '14': 1, '74': 1, '70': 1, '45': 1, '81': 1, '31': 2, '102': 1, '28': 1, '46': 1, '91': 1, '92': 1}
CPA found:  (58, 6)
Still unmatched:  (967, 5)


,Source,Comma,Reference,Question id,Question plh,Law text
0,c.p.a.,None,80,2,Ai sensi dell'{PLH} come avviene la prosecuzio...,None
1,c.p.a.,None,42,3,Entro quale termine le parti devono proporre r...,None
2,c.p.a.,None,133,4,"A norma di quanto dispone l'{PLH}, salvo ulter...",None
3,c.p.a.,None,7,9,Ai sensi dell'{PLH} la giurisdizione amministr...,None
4,c.p.a.,None,34,10,Le pronunce definitive del giudice possono ess...,None


## Extraction of Procedural Penal Code's References

In [110]:
patterns = [
    r'[Aa]rtt?\.( \d+).{0,25}[Cc]\.p\.p[\.]?',
    r'articolo( \d+).{0,25}[Cc]\.p\.p[\.]?',
    r'(?:[Aa]rticolo|[Aa]rtt?\.)( \d+).{0,30} del [Cc]odice di [Pp]rocedura [Pp]enale'
]

df_cpp, df_quiz = extract_data(df_quiz, patterns, "c.p.p.")

print("CPP found: ", df_cpp.shape)
print("Still unmatched: ", df_quiz.shape)
df_cpp.head()

{'57': 2, '59': 1, '71': 3, '78': 2, '83': 2, '84': 2, '90': 5, '96': 1, '97': 2, '102': 1, '103': 5, '104': 1, '110': 1, '111': 1, '112': 1, '116': 1, '118': 1, '120': 1, '122': 2, '124': 1, '125': 1, '133': 2, '134': 1, '136': 1, '137': 2, '141': 1, '142': 1, '149': 1, '154': 2, '156': 2, '159': 1, '161': 1, '166': 1, '171': 1, '172': 1, '183': 1, '190': 1, '197': 1, '198': 1, '199': 1, '200': 1, '202': 1, '220': 1, '221': 1, '222': 1, '245': 2, '247': 2, '260': 3, '261': 1, '262': 1, '316': 1, '324': 2, '350': 1, '352': 2, '353': 2, '355': 1, '357': 2, '374': 1, '376': 1, '380': 3, '381': 2, '386': 1, '387': 1, '451': 1, '558': 1, '662': 1, '55': 2, '56': 1, '46': 2, '52': 2, '36': 2, '37': 1, '61': 1, '148': 1, '258': 1, '289': 1, '321': 2, '322': 1, '354': 1, '740': 2, '745': 1, '391': 2, '410': 1, '414': 1, '415': 1, '460': 2, '369': 3, '468': 1, '45': 1, '585': 1, '464': 1, '98': 1, '309': 1, '593': 1, '444': 7, '610': 1, '191': 1, '129': 2, '329': 1, '275': 1, '93': 1, '666': 1

,law_source,law_number,quiz_id,question_plh,law_text
0,c.p.p.,57,270,Quali tra questi NON sono agenti di polizia gi...,Sono altresì ufficiali e agenti di polizia giu...
1,c.p.p.,57,271,Quali tra questi NON sono ufficiali di polizia...,Sono altresì ufficiali e agenti di polizia giu...
2,c.p.p.,59,272,L'ufficiale preposto ai servizi di polizia giu...,Le sezioni di polizia giudiziaria dipendono da...
3,c.p.p.,71,273,Se ex {PLH} risulta che lo stato mentale dell'...,"Se, a seguito degli accertamenti previsti dall..."
4,c.p.p.,71,274,Con l'ordinanza di sospensione del procediment...,"Se, a seguito degli accertamenti previsti dall..."


## Extraction of Costitution's References

In [93]:
patterns = [
    r'[Aa]rtt?\.?( \d+).{0,25}[Cc]ost'
]

df_cost, df_quiz = extract_data(df_quiz, patterns, "constitution")

print("Costitution found: ", df_cost.shape)
print("Still unmatched: ", df_quiz.shape)
df_cost.head()

 117
 25
 117
 118
 126
 117
 74
 16
 42
 117
 120
 118
 95
 120
 51
 81
 75
 119
 119
 81
 119
 119
 119
 119
 71
 71
 97
 73
 70
 87
{'117': 4, '25': 1, '118': 2, '126': 1, '74': 1, '16': 1, '42': 1, '120': 2, '95': 1, '51': 1, '81': 2, '75': 1, '119': 6, '71': 2, '97': 1, '73': 1, '70': 1, '87': 1}
Costitution found:  (30, 6)
Still unmatched:  (732, 5)


,Source,Comma,Reference,Question id,Question plh,Law text
0,constitution,None,117,5,Consacrando a livello costituzionale i princip...,None
1,constitution,None,25,461,L'{PLH}ituzione disponendo che nessuno puÃ² es...,None
2,constitution,None,117,616,Consacrando a livello costituzionale i princip...,None
3,constitution,None,118,619,Indicare quale principio costituz ionale della...,None
4,constitution,None,126,631,Qualora con decreto motivato e nei casi previs...,None


## Extraction of Legislative Decree's References (NEED TO CATCH ALSO THE REFERENCE ARTICLE)

In [94]:
patterns = [
    r'decreto legislativo n\.( ?\d+ del \d+)',
    r'[Aa]rtt?\.?( \d+).{0,25}D\.Lgs\.(?: n\.)?( \d+\/\d+)',
    r'[Aa]rtt?\.?( \d+).{0,25}D\.Lgs\. n\.( \d+ del \d+)',
    r'D\.Lgs\.(?: n\.)?( \d+\/ ?\d+)',
]

df_dlgs, df_quiz = extract_data(df_quiz, patterns, "D. Lgs.")

print("Laws found: ", df_dlgs.shape)
print("Still unmatched: ", df_quiz.shape)
df_dlgs.head()

{}
Laws found:  (263, 6)
Still unmatched:  (479, 5)


,Source,Comma,Reference,Question id,Question plh,Law text
0,33/2013,None,5,6,L,None
1,50/2016,None,23,14,A,None
2,165/2001,None,34,24,C,None
3,165/2001,None,33,26,D,None
4,165/2001,1,16,28,I,None


## Extraction of Laws References

In [95]:
patterns = [
    r'[Aa]rtt?\.?( \d+).{0,25}[lL]\. ?n\.( \d+\/\d+)',
    r'[Aa]rtt?\.?( \d+).{0,25}[lL]\.( \d+\/\d+)',
    r'n\.( \d+\/\d+)',
    r'[Ll]egge( \d+\/\d+)',
    r'l\.( \d+\/\d+)',
]

def custom_match(match):
    article = match.group(1)
    comma = match.group(2)
    return article, comma

df_laws, df_quiz = extract_data(df_quiz, patterns, "Legge")

print("Laws found: ", df_laws.shape)
print("Still unmatched: ", df_quiz.shape)
df_laws.head()

 241/1990
 241/1990
Legge
(' 9', ' 400/1988')


AttributeError: 'tuple' object has no attribute 'strip'

## Check Missing Rows

In [54]:
# Print Question column of elements that were not matched
for i, row in df_quiz.iterrows():
    print(row['question'])

Il Capo II della l. n. 241/1990 è riservato alla regolamentazione della figura del responsabile del procedimento, ovvero del soggetto al quale è affidato il delicato ruolo di autorità di guida di ciascun procedimento amministrativo. Esso: 
Ai sensi dell'art. 80 C.p.a. come avviene la prosecuzione del giudizio in caso di sospensione? 
Entro quale termine le parti devono proporre ricorso incidentale nell'ambito del processo amministrativo ai sensi dell'art. 42 C.p.a.? 
A norma di quanto dispone l'art. 133 del C.p.a., salvo ulteriori previsioni di legge, a chi sono devolute le controversie aventi ad oggetto i provvedimenti relativi alla disciplina o al divieto dell'esercizio d'industrie insalubri o pericolose? 
Nel procedimento amministrativo, cosa prevede la Legge 241/1990, per come recentemente novellata dalla legge 69/2009, in caso di decorrenza del previsto termine senza che sia stato comunicato il parere obbligatorio di un organo consultivo o senza che esso abbia rappresentato esigen

## Export the data

In [111]:
# Merge the dataframes and clean up the data
df_merged = pd.concat([df_cp, df_cpa, df_cpp, df_cost, df_dlgs, df_laws], ignore_index=True)
print(df_merged.columns)#Source', 'Comma', 'law_number', 'Question id', 'Question plh','Law text
#df_merged.drop(columns=['law_source', 'law_number', 'law_text', 'year'], inplace=True)
df_merged.to_csv(REFERENCES_CSV, index=False)

#print(df_merged.shape)
#print(df_merged.head())

# Generate the quizzes csv with plhs
df_merged = df_merged.dropna(subset=['question_plh'])
df_merged = df_merged.drop_duplicates(subset=['question_plh'])

df_quiz = pd.read_csv(DEFAULT_SAVE_DIR + '/quiz_merged.csv')
df_quiz_plh = pd.merge(df_merged, df_quiz, left_on='quiz_id', right_on='quiz_id', how='inner')
df_quiz_plh.rename(columns={'quiz_id': 'reference_id'}, inplace=True)

print(df_quiz_plh.shape)
print(df_quiz_plh.head())

df_quiz_plh = df_quiz_plh.sort_values(by='reference_id')
df_quiz_plh.to_csv(DEFAULT_SAVE_DIR + '/quiz_merged_plh.csv', index=False)

Index(['law_source', 'law_number', 'quiz_id', 'question_plh', 'law_text',
       'year'],
      dtype='object')
(357, 10)
  law_source law_number  reference_id  \
0       c.p.        240         224.0   
1       c.p.        266         225.0   
2       c.p.         24         226.0   
3       c.p.          7         227.0   
4       c.p.         19         228.0   

                                        question_plh  \
0  Ai sensi dell'{PLH}Ã¨ sempre ordinata la confi...   
1  Ai sensi dell'{PLH}Ã¨ punito, se il fatto non ...   
2  La pena della multa ex {PLH}consiste nel pagam...   
3  Ai sensi dell'{PLH}Ã¨ punito secondo la legge ...   
4  Tra le pene accessorie per i delitti ex {PLH}t...   

                                            law_text  year  \
0  (Confisca)  Nel caso di condanna, il giudice p...   NaN   
1  (Istigazione di militari a disobbedire alle le...   NaN   
2  (Multa)  La pena della multa consiste nel paga...   NaN   
3  (Reati commessi all'estero)  È punito secon

## Extraction of a Queries-Pos-Neg dataset

In [6]:
df_quiz = pd.read_csv(QUIZZES_CSV)
df_references = pd.read_csv(REFERENCES_CSV)
df_laws = pd.read_csv(LAWS_CSV)
dataset = [] # query, pos, neg

print(df_quiz.columns)#'quiz_id', 'question', 'answer_1', 'answer_2', 'answer_3'
print(df_references.columns)#Source', 'Comma', 'law_number', 'Question id', 'Question plh','Law text
print(df_laws.columns)#law_source,year,law_number,law_text

# save in a file Source and Comma fields where Law text is nan
missing = df_references.loc[df_references['law_text'].isna() & ((df_references["law_source"] == "c.p.") | (df_references["law_source"] == "c.p.p.")), ['law_source', 'law_number']]
missing.to_csv(DEFAULT_SAVE_DIR + '/missing_laws.csv', index=False)

print(missing)

for i, row in df_references.iterrows():
    query = df_quiz.loc[df_quiz['quiz_id'] == row['quiz_id'], 'question']
    if query.empty:
        continue
    query = query.iloc[0]
    pos = row['law_text']
    
    if pd.isna(pos):
        continue
    
    for _ in range(3):
        neg = pd.NA
        while pd.isna(neg):
            neg = df_laws.sample(1)['law_text'].iloc[0]
        dataset.append({
            'query': query,
            'pos': pos,
            'neg': neg
        })
print(len(dataset))
print(dataset[-1])
print(dataset[-2])
print(dataset[-3])

with open(DEFAULT_SAVE_DIR + '/training_data.json', 'w+') as f:
    json.dump(dataset, f)

Index(['quiz_id', 'question', 'answer_1', 'answer_2', 'answer_3'], dtype='object')
Index(['law_source', 'law_number', 'quiz_id', 'question_plh', 'law_text',
       'year'],
      dtype='object')
Index(['law_source', 'year', 'law_number', 'law_text'], dtype='object')
     law_source law_number
141        c.p.        283
152        c.p.        295
1943     c.p.p.   Art. 560
1944     c.p.p.   Art. 561
1945     c.p.p.   Art. 562
1946     c.p.p.   Art. 563
1947     c.p.p.   Art. 564
1948     c.p.p.   Art. 565
1949     c.p.p.   Art. 566
1950     c.p.p.   Art. 567
2141     c.p.p.   Art. 728
1068
{'query': "L'articolo 4 del Codice di procedura penale afferma che per determinare la competenza del giudice penale si ha riguardo:", 'pos': 'Per determinare la competenza si ha riguardo alla pena stabilita dalla legge per ciascun reato consumato o tentato. Non si tiene conto della continuazione, della recidiva e delle circostanze del reato, fatta eccezione delle circostanze aggravanti per le quali la